# Notebook regarding measurements with RNNs

## Saving the dataset

In [1]:
import datautils
import torch
from torch.utils.data import DataLoader
import pickle
import torch.nn as nn
import RNNutils
import time
import pandas as pd
from torchtext.data import get_tokenizer

c:\Users\mirda\OneDrive\Desktop\Group4-project\hltenv\Lib\site-packages\torchtext\vocab\__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
c:\Users\mirda\OneDrive\Desktop\Group4-project\hltenv\Lib\site-packages\torchtext\utils.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
c:\Users\mirda\OneDrive\Desktop\Group4-project\hltenv\Lib\site-packages\torchtext\data\__init

In [2]:
DATASET = "./Dataset/power-gb-train.tsv"
DATA_DIR = "./Dataset/"
RES_DIR = "./Results/"
EMBED_DIR = "./Embeddings/"
CHECK_DIR = "./Checkpoints/"
DEVICE = datautils.get_device()

load the dataset (with hold-out)

In [7]:
from collections import Counter


X_train, y_train, X_val, y_val, X_test, y_test = (
    datautils.split_holdout_dataset(DATASET)
)

# get tokenizer
tokenizer = get_tokenizer("spacy", language="en_core_web_sm")

# build vocabulary
min_freq = 1
curr_vocab = None
while curr_vocab is None or len(curr_vocab) > 10000:
    curr_vocab = datautils.build_vocab(X_train, tokenizer, min_freq=min_freq)
    curr_vocab.set_default_index(curr_vocab["<unk>"])
    min_freq += 1
    print(len(curr_vocab), f"{min_freq} min_freq")

# process datasets
X_train = datautils.data_process(X_train, curr_vocab, tokenizer)
X_val = datautils.data_process(X_val, curr_vocab, tokenizer)
X_test = datautils.data_process(X_test, curr_vocab, tokenizer)

# create dataset objects
X_train = datautils.TextDataset(X_train, y_train, curr_vocab)
X_val = datautils.TextDataset(X_val, y_val, curr_vocab)
X_test = datautils.TextDataset(X_test, y_test, curr_vocab)

# save datasets
torch.save(X_train, f"{DATA_DIR}train_dataset.pt")
torch.save(X_val, f"{DATA_DIR}val_dataset.pt")
torch.save(X_test, f"{DATA_DIR}test_dataset.pt")

# save vocabulary with pickle
with open(f"{EMBED_DIR}vocab.pkl", "wb") as f:
    pickle.dump(curr_vocab, f)

72276 2 min_freq
46491 3 min_freq
37987 4 min_freq
33080 5 min_freq
29729 6 min_freq
27245 7 min_freq
25353 8 min_freq
23816 9 min_freq
22555 10 min_freq
21529 11 min_freq
20552 12 min_freq
19712 13 min_freq
18970 14 min_freq
18299 15 min_freq
17669 16 min_freq
17070 17 min_freq
16575 18 min_freq
16108 19 min_freq
15623 20 min_freq
15203 21 min_freq
14864 22 min_freq
14495 23 min_freq
14173 24 min_freq
13883 25 min_freq
13564 26 min_freq
13267 27 min_freq
12996 28 min_freq
12756 29 min_freq
12553 30 min_freq
12356 31 min_freq
12150 32 min_freq
11959 33 min_freq
11747 34 min_freq
11550 35 min_freq
11374 36 min_freq
11174 37 min_freq
10984 38 min_freq
10830 39 min_freq
10679 40 min_freq
10514 41 min_freq
10381 42 min_freq
10228 43 min_freq
10092 44 min_freq
9944 45 min_freq


Please execute this cell only when you want to use pretrained embeddings, otherwise skip if you don't intend to.

In [3]:
from gensim.downloader import load

EMBEDDING = "word2vec-ruscorpora-300"
# download the embedding
pretrained_embeddings = load(EMBEDDING)
vector_embeddings = None
with open(f"{EMBED_DIR}vocab.pkl", "rb") as f:
    curr_vocab = pickle.load(f)
    # Create a matrix to store the vectors
    vector_embeddings = torch.zeros(
        len(curr_vocab), pretrained_embeddings.vector_size
    )

    # Set the vectors for our vocabulary words
    for i, word in enumerate(curr_vocab.get_itos()):
        if word in pretrained_embeddings:
            vector_embeddings[i] = torch.tensor(pretrained_embeddings[word])

### Training hold-out validation

In [4]:
import pandas as pd
import itertools

In [5]:
# load the datasets
train_dataset = torch.load(f"{DATA_DIR}train_dataset.pt")
val_dataset = torch.load(f"{DATA_DIR}val_dataset.pt")

# load the vocabulary
with open(f"{EMBED_DIR}vocab.pkl", "rb") as f:
    curr_vocab = pickle.load(f)

In [6]:
N_EPOCHS = 1000
CLIP = 1
BATCH_SIZE = 8
EMBEDDING_DIM = 128
HIDDEN_DIM = 1024
OUTPUT_DIM = 1
INPUT_DIM = len(curr_vocab)
PATIECE = 10
CHECKPOINT_STEPS = 10
DROP_OUT = 0.2
LAYERS = 3
MEAN_POOL = True
SEED = 0
# experiment number
RES_NUM = 12


learning_rate_list = [0.001]
weight_decay_list = [0]

If you wish to use a pretrainded model, please set the embedding dimension to the same size as pretrained embedding and execute the cell necessary to load it.

In [7]:
torch.manual_seed(SEED)
model = RNNutils.BiLSTM(
    INPUT_DIM,
    300,  # set to the same size as the pretrained embeddings, else use EMBEDDING_DIM
    HIDDEN_DIM,
    output_dim=OUTPUT_DIM,
    dropout=DROP_OUT,
    n_layers=LAYERS,
    mean_pooling=MEAN_POOL,
    pretrained_embedding=vector_embeddings,  # leave to None for no pretrained embeddings
    device=DEVICE,
).to(DEVICE)

train_iterator = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    collate_fn=train_dataset.generate_batch,
)
val_iterator = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    collate_fn=val_dataset.generate_batch,
)
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate_list[0])

In [8]:
model

BiLSTM(
  (embedding): Embedding(9944, 300)
  (lstm): LSTM(300, 1024, batch_first=True, bidirectional=True)
  (fc): Sequential(
    (0): Linear(in_features=2048, out_features=1024, bias=True)
    (1): Dropout(p=0.2, inplace=False)
    (2): Sigmoid()
    (3): Linear(in_features=1024, out_features=1024, bias=True)
    (4): Dropout(p=0.2, inplace=False)
    (5): Sigmoid()
    (6): Linear(in_features=1024, out_features=1, bias=True)
  )
  (dropout): Dropout(p=0.2, inplace=False)
  (activation): Sigmoid()
)

In [9]:
min_loss = float("inf")
patience = PATIECE

# initailize dataframe for results
results = pd.DataFrame(
    columns=[
        "optim",
        "lr",
        "weight_decay",
        "epoch",
        "train_loss",
        "val_loss",
        "precision",
        "recall",
        "f1_score",
    ]
)

for lr, weight_decay in itertools.product(
    learning_rate_list, weight_decay_list
):

    for epoch in range(N_EPOCHS):

        start_time = time.process_time()

        print("Training...")
        train_loss = RNNutils.train_rnn(
            model,
            train_iterator,
            optimizer,
            criterion,
            CLIP,  # device=DEVICE
            device=DEVICE,
        )
        print("Evaluating...")
        valid_loss, precision, recall, f1_score = RNNutils.evaluate(
            model,
            val_iterator,
            criterion,  # device=DEVICE
            device=DEVICE,
        )

        end_time = time.process_time()

        results = pd.concat(
            [
                results,
                pd.DataFrame(
                    {
                        "optim": "Adam",
                        "lr": lr,
                        "weight_decay": 0,
                        "epoch": [epoch + 1],
                        "train_loss": [train_loss],
                        "val_loss": [valid_loss],
                        "precision": [precision],
                        "recall": [recall],
                        "f1_score": [f1_score],
                    }
                ),
            ]
        )

        # save model checkpoint every 10 epochs
        if (epoch + 1) % CHECKPOINT_STEPS == 0:
            torch.save(
                {
                    "epoch": epoch,
                    "model_state_dict": model.state_dict(),
                    "optimizer_state_dict": optimizer.state_dict(),
                    "loss": valid_loss,
                    "patience": patience,
                },
                f"{RES_DIR}rnn_checkpoint_{epoch + 1}.pt",
            )
            results.to_csv(
                f"{RES_DIR}rnn_results-{RES_NUM}-temp.csv", index=False
            )

        print(f"Epoch: {epoch+1} | Time: {end_time-start_time:.2f}s")
        print(f"\tTrain Loss: {train_loss:.3f}")
        print(f"\t Val. Loss: {valid_loss:.3f}")
        print(
            f"\t Val. Precision: {precision:.3f}, Recall: {recall:.3f}, F1 Score: {f1_score:.3f}"
        )

        # early stopping
        if valid_loss < min_loss:
            min_loss = valid_loss
            patience = PATIECE
        else:
            patience -= 1

        if patience == 0:
            break
results.to_csv(f"{RES_DIR}bilstm-results{RES_NUM}.csv", index=False)

Training...
Evaluating...


c:\Users\mirda\OneDrive\Desktop\Group4-project\hltenv\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\mirda\AppData\Local\Temp\ipykernel_27660\756629584.py:46: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results = pd.concat(


Epoch: 1 | Time: 75.02s
	Train Loss: 0.704
	 Val. Loss: 0.713
	 Val. Precision: 0.284, Recall: 0.500, F1 Score: 0.362
Training...
Evaluating...


c:\Users\mirda\OneDrive\Desktop\Group4-project\hltenv\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Epoch: 2 | Time: 65.52s
	Train Loss: 0.701
	 Val. Loss: 0.695
	 Val. Precision: 0.284, Recall: 0.500, F1 Score: 0.362
Training...
Evaluating...


...

c:\Users\mirda\OneDrive\Desktop\Group4-project\hltenv\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Epoch: 19 | Time: 71.03s
	Train Loss: 0.704
	 Val. Loss: 0.689
	 Val. Precision: 0.284, Recall: 0.500, F1 Score: 0.362
Training...
Evaluating...


c:\Users\mirda\OneDrive\Desktop\Group4-project\hltenv\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Epoch: 20 | Time: 68.98s
	Train Loss: 0.703
	 Val. Loss: 0.703
	 Val. Precision: 0.216, Recall: 0.500, F1 Score: 0.302
Training...
Evaluating...
Epoch: 21 | Time: 65.44s
	Train Loss: 0.703
	 Val. Loss: 0.695
	 Val. Precision: 0.216, Recall: 0.500, F1 Score: 0.302


c:\Users\mirda\OneDrive\Desktop\Group4-project\hltenv\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [11]:
results = pd.read_csv(f"{RES_DIR}bilstm-results{RES_NUM}.csv")
results.sort_values("f1_score", inplace=True)
results

,optim,lr,weight_decay,epoch,train_loss,val_loss,precision,recall,f1_score
20,Adam,0.001,0,21,0.702761,0.694779,0.215968,0.5,0.301645
6,Adam,0.001,0,7,0.702255,0.707960,0.215968,0.5,0.301645
13,Adam,0.001,0,14,0.703440,0.693417,0.215968,0.5,0.301645
11,Adam,0.001,0,12,0.706107,0.695673,0.215968,0.5,0.301645
19,Adam,0.001,0,20,0.702863,0.703047,0.215968,0.5,0.301645
18,Adam,0.001,0,19,0.703807,0.689012,0.284032,0.5,0.362271
17,Adam,0.001,0,18,0.701934,0.686279,0.284032,0.5,0.362271
16,Adam,0.001,0,17,0.703139,0.691401,0.284032,0.5,0.362271
15,Adam,0.001,0,16,0.704091,0.688685,0.284032,0.5,0.362271
14,Adam,0.001,0,15,0.706379,0.690040,0.284032,0.5,0.362271


In [13]:
with torch.no_grad():
    test_dataset = torch.load(f"{DATA_DIR}test_dataset.pt")
    test_iterator = DataLoader(
        test_dataset,
        batch_size=BATCH_SIZE,
        collate_fn=train_dataset.generate_batch,
        shuffle=True,
    )
    test_loss, precision, recall, f1_score = RNNutils.evaluate(
        model,
        test_iterator,
        criterion,  # device=DEVICE
        device=DEVICE,
    )
    print(test_loss, precision, recall, f1_score)
    print("confusion matrix")
    print(
        RNNutils.evaluate_confusion_matrix(
            model, val_iterator, criterion, device=DEVICE
        )
    )

c:\Users\mirda\OneDrive\Desktop\Group4-project\hltenv\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


0.6947248345957353 0.21707757065544198 0.5 0.3027253668763103
confusion matrix
(array([[2586,    0],
       [3401,    0]], dtype=int64), 0.21596793051611826, 0.5, 0.3016446984719468)


c:\Users\mirda\OneDrive\Desktop\Group4-project\hltenv\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
